# Rough outline of a method for calorimeter calibration

We present a method to calibrate a sampling calorimeter for a linear collider detector concept. In this method, we assume that a bunch of single particles with different, but well-defined energies, have been shot at the calorimeter.
We further assume two different calibration constants, one for the ECAL, one for the HCAL.

## The setup
First, we load a bunch of useful packages that we'll use in the course of this exercise

In [1]:
using LCIO
using Plots
using Glob
using LsqFit
using StatsBase
using Distributions
gr();

The following reads the files for the single particles. For this, we are using the convenient `readdir` method from the Glob package. The energies are part of the file name and can be extracted with a regex.

In [2]:
# read the files for calibration
fileList = readdir(glob"K0L*barrel*", "SingleParticles")
# build the list of input energies
energyString = r"_(\d+)GeV"
# get (match) the energy string from the filename, convert (parse) from string to Int16, build a dictionary from ints to filenames
energyMap = Dict((parse(Int16, match(energyString, fn)[1]), fn) for fn in fileList)
for (energy, filename) in energyMap
    println(energy, " GeV:\t", filename)
end

20 GeV:	SingleParticles/K0L_20GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio
100 GeV:	SingleParticles/K0L_100GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio
10 GeV:	SingleParticles/K0L_10GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio
50 GeV:	SingleParticles/K0L_50GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio
2 GeV:	SingleParticles/K0L_2GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio
5 GeV:	SingleParticles/K0L_5GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio
1 GeV:	SingleParticles/K0L_1GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio


Now we are ready to read the hits from the files. We are multiplying the higher ecal layers with an intrinsic calibration of 2, since they are behind twice the material of the inner layers. We are also cutting out events that don't have a lot of hits in total, as a method of data cleaning. We return the sum of the hit energies, and the number of hits for ECal and HCal separately.

In [3]:
function getHitsFromFile(filename)
    eCalEnergies = Float64[]
    hCalEnergies = Float64[]
    eCalLengths = Int16[]
    hCalLengths = Int16[]
    LCIO.open(filename) do reader
        for (idx, event) in enumerate(reader)
            if idx > 10000
                break
            end
            hCalEnergy = 0.0
            hCalLength = 0
            eCalEnergy = 0.0
            eCalLength = 0
            # sum up the uncalibrated HCalHits
            for hit in getCollection(event, "HcalBarrelHits")
                hCalEnergy += getEnergy(hit)
                hCalLength += 1
            end
            # sum up the uncalibrated ECalHits
            # this needs to be sorted by layer, so we need a decoder
            EcalBarrelHits = getCollection(event, "EcalBarrelHits")
            decode = CellIDDecoder(EcalBarrelHits)
            for hit in EcalBarrelHits
                # calibrate the hits in the later layers with a higher number,
                # because they are behind thicker tungsten slabs
                factor = decode(hit)["layer"] < 20 ? 1 : 2
                eCalEnergy += factor*getEnergy(hit)
                eCalLength += 1
            end
            # fixme: Simple outlier cut
            if eCalLength+hCalLength < 100
                continue
            end
            push!(eCalEnergies, eCalEnergy)
            push!(eCalLengths, eCalLength)
            push!(hCalEnergies, hCalEnergy)
            push!(hCalLengths, hCalLength)
        end
    end
    return eCalEnergies, eCalLengths, hCalEnergies, hCalLengths
end;

In [4]:
eHits  = Dict{Int16, Vector{Float64}}()
eCount = Dict{Int16, Vector{Int16}}()
hHits  = Dict{Int16, Vector{Float64}}()
hCount = Dict{Int16, Vector{Int16}}()
for (energy, filename) in energyMap
    if energy < 10
        continue
    end
    println("Processing file for ", energy, " GeV")
    eCal, nEhits, hCal, nHhits = getHitsFromFile(filename)
    eHits[energy] = eCal
    eCount[energy] = nEhits
    hHits[energy] = hCal
    hCount[energy] = nHhits
end

Processing file for 20 GeV
Processing file for 100 GeV
Processing file for 10 GeV
Processing file for 50 GeV


## Fitting the calibration constants to the particle energies 

Before going into the fitting, we need a function to cut of tails from the distributions. This will reduce the bias towards lower values due to stochastic fluctuations to lower values.

In [5]:
# removes the 10% of the furthest outliers on either side
# no assumption about smoothness
function removeTails(distribution, cutOff=10)
    sort!(distribution)
    l = length(distribution)
    lcut = round(Int64, l * cutOff/100)
    hcut = round(Int64, l * (100-cutOff)/100)
    # start out with the whole distribution
    minDist = distribution[end] - distribution[1]
    low = 1
    high = l
    for idx = 1:lcut
        dist = distribution[hcut+idx] - distribution[idx]
        if dist < minDist
            minDist = dist
            low = idx+1
            high = hcut+idx
        end
    end
    return low, high
end;

First shot at fitting: Each energy point individually, but using one constant for the ECal, another one for the HCal

In [6]:
function fitter(ecal, hcal, energy)
    function model(x, p)
        return p[1] .* x[1] + p[2] .* x[2]
    end
    fit = curve_fit(model, [ecal, hcal], energy, [50.0, 35.0])
    return fit.param
end
hCalCalibration = Dict{Int16, Float64}()
eCalCalibration = Dict{Int16, Float64}()
for energy in keys(hHits)
    ecal = eHits[energy]
    hcal = hHits[energy]
    calibration = fitter(ecal, hcal, energy)
    eCalCalibration[energy] = calibration[1]
    hCalCalibration[energy] = calibration[2]
    println(energy, " GeV: \t", eCalCalibration[energy], "\t", hCalCalibration[energy])
end

100 GeV: 	50.04390420833081	37.266048754667324
10 GeV: 	51.12434124545007	35.4419737491782
50 GeV: 	49.676409091647365	36.12680269773898
20 GeV: 	49.37054664308238	35.07231766151422


In [7]:
histogram([eCalCalibration[energy] .* eHits[energy] .+ hCalCalibration[energy] .* hHits[energy] for energy in keys(hHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(hHits)))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 10 
 
 
 
 
 50 
 
 
 
 
 20

This looks pretty good, but the fit values are not the same for different energies. We need a method that fits to all energies simultaneously.

## Fitting by optimizing for linearity
To get a measure of global performance, we compute one ECal calibration and one HCal calibration for all energies, and use the value `ratio of calibrated energy over particle energy - 1` as the loss function in the minimization.  

In [8]:
# this function attempts a global fit and minimizes the offset b of the linear form y=mx+b
# parameters are ecal energies (×2 for the hits in the outer layers), hcal energies, particle energy
function lineFitter(ecal, hcal, truValues)
    function model(x, p)
        energies = Dict{Int16, Float64}()
        for e in truValues
            calibrated = p[1] .* x[1][e] + p[2] .* x[2][e]
            # cut the tails, fit a Normal distribution to the result
            low, high = removeTails(calibrated)
            n = Distributions.fit(Normal, calibrated[low:high])
            energies[e] = n.μ
        end
        # we are optimizing for the ratio of reconstructed energies to true values
        return [energies[e]/e for e in truValues]
    end
    fit = curve_fit(model, [ecal, hcal], 1.0, [50.0, 36.0])
    return fit.param
end
ECal, HCal = lineFitter(eHits, hHits, keys(hHits))
println("ECal calibration constant: ", ECal)
println("ECal calibration constant: ", HCal)

ECal calibration constant: 50.068279178715756
ECal calibration constant: 36.01731842347431


## Results
Let's take a look at the distribution of the different energies and how well the mean values line up with the particle energies.

In [9]:
histogram([ECal .* eHits[energy] .+ HCal .* hHits[energy] for energy in keys(hHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(hHits)))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 10 
 
 
 
 
 50 
 
 
 
 
 20

In [10]:
y = Vector{Float64}()
yerr = Vector{Float64}()
x = Vector{Float64}()
for e in keys(hHits)
    d = ECal .* eHits[e] .+ HCal .* hHits[e]
    low, high = removeTails(d)
    gauss = Distributions.fit(Normal, d[low:high])
    push!(y, gauss.μ)
    push!(yerr, gauss.σ)
    push!(x, 1.0*e)
end
line(x, p) = p[1] + p[2]*x
l = curve_fit(line, x, y, [0.0, 1.0])
plot(x, y, yerr=yerr, marker=stroke(2), line=false, label="data")
leg = @sprintf("y=%.2f + %.2f*x", l.param[1], l.param[2])
plot!(x, l.param[1]+l.param[2]*x, label=leg)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 y=0.56 + 0.98*x